In [1]:
import sys, os, re
import numpy as np
from scipy.spatial import distance
from scipy.spatial.distance import cdist
import pandas as pd
from rdkit import Chem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import fileinput
from pharma import pharma
import featureSASA
import xgboost as xgb
import pickle
from openbabel import openbabel as ob
from openbabel import pybel
import alphaspace2 as al
import mdtraj

In [2]:
import calc_bridge_wat
import calc_ligCover_betaScore
import calc_rdkit
import calc_sasa
import calc_vina_features
import prepare_betaAtoms

In [3]:
Vina = '/home/cyang/paper_XGB/delta_LinF9_XGB/software/smina_feature'
Smina = '/home/cyang/paper_XGB/delta_LinF9_XGB/software/smina.static'
SF = '/home/cyang/paper_XGB/delta_LinF9_XGB/software/sf_vina.txt'
ADT = '/home/cyang/MGLTools-1.5.6/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py'

#### Input

In [4]:
pro = '../test/1e66_wat/1e66_protein.pdb'
lig = '../test/1e66_wat/1e66_ligand.mol2'

In [5]:
if lig.endswith('.mol2'):
    mol = Chem.MolFromMol2File(lig, removeHs=False)
    lig = lig[:-5]+'.pdb'
    Chem.MolToPDBFile(mol, lig)
        
elif lig.endswith('.sdf'):
    mol = Chem.MolFromMolFile(lig, removeHs=False)
    lig = lig[:-4]+'.pdb'
    Chem.MolToPDBFile(mol, lig)
        
elif lig.endswith('.pdb'):
    mol = Chem.MolFromPDBFile(lig, removeHs=False)

#### prepare_betaAtoms

In [6]:
beta = os.path.join(os.path.dirname(pro), 'betaAtoms.pdb')
pro_pdbqt = prepare_betaAtoms.Prepare_beta(pro, beta, ADT)
pro_pdbqt

setting PYTHONHOME environment
adding gasteiger charges to peptide
Sorry, there are no Gasteiger parameters available for atom 1e66_protein:A:GLU247:OE1
Sorry, there are no Gasteiger parameters available for atom 1e66_protein:A:GLU247:OE2
Sorry, there are no Gasteiger parameters available for atom 1e66_protein:B:GLU247:OE1
Sorry, there are no Gasteiger parameters available for atom 1e66_protein:B:GLU247:OE2
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH1:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH2:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH3:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH4:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH5:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH6:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH7:O
Sorry, there ar

Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH973:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH974:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH975:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH976:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH977:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH978:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH979:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH980:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH981:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH982:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :HOH983:O
Sorry, there are no Gasteiger parameters available for atom 1e66_protein: :H

'../test/1e66_wat/1e66_protein.pdbqt'

#### Vina_features

In [7]:
v = calc_vina_features.vina(pro_pdbqt, lig, Vina, Smina)
vinaF = [v.LinF9]+v.features(48)

In [8]:
len(vinaF)

49

#### Beta_features

In [9]:
betaScore, ligCover = calc_ligCover_betaScore.calc_betaScore_and_ligCover(lig, beta)
betaF = [betaScore, ligCover]
betaF

[-15.3, 0.952]

#### sasa_features

In [10]:
datadir = os.path.dirname(os.path.abspath(pro))
pro_ = os.path.abspath(pro)
lig_ = os.path.abspath(lig)
sasa_features = calc_sasa.sasa(datadir,pro_,lig_)

In [11]:
sasaF = sasa_features.sasa+sasa_features.sasa_lig+sasa_features.sasa_pro
len(sasaF)

30

#### ligand_features

In [12]:
mol = Chem.MolFromPDBFile(lig, removeHs=False)
ligF = list(calc_rdkit.GetRDKitDescriptors(mol))
len(ligF)

10

#### water_features

In [13]:
df = calc_bridge_wat.Check_bridge_water(pro, lig)
if len(df) == 0:
    watF = [0,0,0]
else:
    Nbw, Epw, Elw = calc_bridge_wat.Sum_score(pro, lig, df, Smina)
    watF = [Nbw, Epw, Elw]

In [14]:
len(watF)

3

#### Calculate XGB

In [16]:
LinF9 = vinaF[0]*(-0.73349)
LE = LinF9/vinaF[-4]
sasa = sasaF[:18]+sasaF[19:28]+sasaF[29:]
metal = vinaF[1:7]
X = vinaF[7:]+[ligCover,betaScore,LE]+sasa+metal+ligF+watF
X = np.array([X]).astype(np.float64)

In [17]:
y_predict_ = []
for i in range(1,11):
    xgb_model = pickle.load(open("/home/cyang/paper_XGB/delta_LinF9_XGB/saved_model/mod_%d.pickle.dat"%i,"rb"))
    y_i_predict = xgb_model.predict(X, ntree_limit=xgb_model.best_ntree_limit)
    y_predict_.append(y_i_predict)

In [18]:
y_predict = np.average(y_predict_, axis=0)
XGB = round(y_predict[0]+LinF9,3)

In [19]:
XGB

8.893